# UBC Library - LOCR Research
## Search and Data Mining Tools
Created by [Neil Aitken](https://neil.aitken.com)

In the past, it was hard to know which texts were already in LOCR (UBC's digital materials online reserve system) or to gather information about what materials have historically been used for particular classes.

Now, provided we have access to an Excel or CSV file version of the LOCR report in Tableau, we can conduct research and generate more specific reports to aid with our course planning, syllabus development, and collection development.


### Import code libraries for file access, csv, and data parsing and analysis

In [1]:
import os
import csv
import pandas as pd

### Upload a data file containing LOCR info (CSV or Excel format)

In [2]:
from google.colab import files
uploaded = files.upload()

Saving LOCR_items_data_v2.xlsx to LOCR_items_data_v2.xlsx


### Define load functions: *load_csv()* and *load_excel()*

In [3]:
# Load csv file and returns dataframe
def load_csv(fname):
   import pandas as pd
   return pd.read_csv(fname)

# Load excel file and returns dataframe
def load_excel(fname):
  import pandas as pd
  return pd.read_excel(fname)

# Load file and determine if Excel or csv, then return dataframe
def load_datafile(fname):
  start = len(fname)-5
  if start < 0:
    start = 0
  num_periods = fname.count('.',start,len(fname))
  if num_periods == 1:
    name,extension = fname.split('.')
    if extension == 'xlsx' or extension == 'xls':
      print('Excel sheet')
      return load_excel(fname)
    else:
      print('CSV doc')
      return load_csv(fname)
  elif num_periods == 0:
    print('Filename should have a .csv, .txt, or .xlsx extension')
  else:
    print('Filename is improperly formatted. Please rename and ensure it is a CSV or Excel file')
  return None



In [4]:
class Record:
    def __init__(self,keys,datalist):
        i = 0
        self.keys = keys

        for val in datalist:
            setattr(self,str(keys[i]),val)
            i+=1

        # Each entry in the course reserves data has a course code and course number stored separately.
        # We combine them to form the course ID and store it in the record as a new field to save time
        try:
          self.course_id = str(getattr(self,'Course code'))+' '+str(self.Coursenumber)
        except:
          print(f'Keys = {keys}')

    def read(self,headers,series):
      for hd in headers:
        setattr(self,hd,series[hd])

    def get(self,rkey='all'):
        if rkey == 'all' or not rkey in vars(self):
            result =''
            for key in vars(self):
                result = result +'\n' + getattr(self,key)
            return result
        else:
            return getattr(self,rkey)

    def has_str(self,searchstr):
        import re
        for key in vars(self):
            strmatch = re.search(searchstr,self.get(key))
            if not strmatch==None:
                return True
        return False

    def dump(self,rkey='all'):
        if rkey=='brief':
            result = '"'+str(self.Title)+'," '+str(self.Author)+', '+str(getattr(self,'Call number'))+' '+str(self.course_id)
        elif rkey == 'all' or not rkey in vars(self):
            for key in vars(self):
                if not key == 'keys':
                    result = key+': '+str(getattr(self,key))
        elif rkey=='keys':
            result = self.keys
        else:
          result = rkey+': '+str(getattr(self,rkey))
        return result

    def print(self,rkey='all'):
      print(self.dump(rkey))

class Catalog:
    def __init__(self,df):
        self.dataframe = df
        self.catalog = []
        self.keys = df.columns
        for ind in range(0,len(df)):
             record = Record(self.keys,list(df.loc[ind]))
             self.catalog.append(record)

    def print(self,key='all'):
        for ind in range(0,len(self.catalog)):
            print(self.catalog[ind].print(key))

    def get(self):
        return self.catalog

    def search(self,key,value):
        import re
        results = []
        if value == '*':
          return self.catalog

        for record in self.catalog:
            if key in vars(record):
                occ = re.search(value.lower(),str(getattr(record,key)).lower())
                if not occ == None:
                    results.append(record)
            else:
              for k in vars(record):
                occ = re.search(value.lower(),str(getattr(record,k)).lower())
                if not occ == None:
                    results.append(record)

        return results

    def search_by_course_id(self,courses):
      import re
      results = []
      course_list = courses.split('|')

      for record in self.catalog:
        for course in course_list:
          if record.course_id == course:
            results.append(record)

      return results


### Load Data File
Replace 'LOCR_items_data_v2.xlsx' with name of current data file.

The function will take both Excel files (.xls, .xlsx) and CSV files (.csv,.txt).

In [6]:
df = load_datafile('LOCR_items_data_v2.xlsx')

Excel sheet


## Gathering Information

### Get Information about Data Set

In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 19 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Title                                     2864 non-null   object        
 1   Author                                    2863 non-null   object        
 2   ISXN                                      1906 non-null   object        
 3   Publisher                                 2400 non-null   object        
 4   Uri                                       2769 non-null   object        
 5   Call number                               2377 non-null   object        
 6   Item type                                 2864 non-null   object        
 7   Bib ID from resolved URI or Filelocation  175 non-null    float64       
 8   Shorturl                                  2864 non-null   object        
 9   Branch                        

### Get List of Column Names
You can use these names to finetune your searches.

In [8]:
print(df.columns)

Index(['Title', 'Author', 'ISXN', 'Publisher', 'Uri', 'Call number',
       'Item type', 'Bib ID from resolved URI or Filelocation', 'Shorturl',
       'Branch', 'Session', 'Course code', 'Coursenumber', 'Lmsid', 'Section',
       'Course title', 'Physical Format', 'Status in LOCR',
       'Day of Last Status Change'],
      dtype='object')


### Define Function: Search by Keyword
This function allows you to search for a key word, phrase, or a list of phrases.

1. Search for a single term
  - search_by_keyword('dog')
2. Search for multiple terms
  - search_by_keyword('song|novel|war')
  

In [9]:
catalog = Catalog(df)
keywords = 'monster|science fiction|fantasy|speculative fiction|horror|weird|spec fic|fairy tale|paranormal|robot|automaton|automata|artificial intelligence|future|climate fiction|dystopian|apocalypse|apocalyptic'

results = catalog.search('Title',keywords)
for result in results:
  result.print('brief')


"A child's work: The importance of fantasy play," Paley, Vivian Gussin, nan EDCP 508A
"A child's work: The importance of fantasy play," Paley, Vivian Gussin, nan EDCP 508A
"A child's work: the importance of fantasy play," Paley, Vivian Gussin, LB1139.35.P55 P35 2004 EDCP 508A
"American horror story: The complete third season / Coven," Farmiga, Taissa;Buecker, Bradley;Conroy, Frances;Bates, Kathy;Gomez-Rejon, Alfonso;Lange, Jessica;Peters, Evan;Rabe, Lily;Roberts, Emma;O'Hare, Denis;Deutch, Howard;Paulson, Sarah;Bassett, Angela;Murphy, Ryan;Sidibe, Gabourey;Falchuk, Brad;Uppendahl, Michael;Podeswa, Jeremy, PN1992.77 .A555 2014 dvd CENS 307
"American horror story: The complete third season / Coven," Farmiga, Taissa;Buecker, Bradley;Conroy, Frances;Bates, Kathy;Gomez-Rejon, Alfonso;Lange, Jessica;Peters, Evan;Rabe, Lily;Roberts, Emma;O'Hare, Denis;Deutch, Howard;Paulson, Sarah;Bassett, Angela;Murphy, Ryan;Sidibe, Gabourey;Falchuk, Brad;Uppendahl, Michael;Podeswa, Jeremy, PN1992.77 .A555 2

In [10]:
def search_by_keyword(searchstr):
  import re
  results = []
  for ind in range(0,len(df)):
    row = list(df.loc[ind])
    bFound = False
    for i in range(0,len(row)):
      test = re.search(searchstr,str(row[i]).lower())
      if not test == None:
        bFound = True
    if bFound:
      entry = Record(df.loc[ind].index,row)
      results.append(entry)
  for ind in range(0,len(results)):
    results[ind].print('brief')






### Search by Keyword
Enter your search terms in the **keywords** field

In [11]:
keywords = 'monster|science fiction|fantasy|speculative fiction|horror|weird|spec fic|fairy tale|paranormal|robot|automaton|automata|artificial intelligence|future|climate fiction|dystopian|apocalypse|apocalyptic'
search_by_keyword('science fiction')



"Archaeologies of the future: the desire called utopia and other science fictions," Jameson, Fredric, PS648.S3 J36 2007 ENGL 505A
"Colonialism and the emergence of science fiction," Rieder, John, PS374.S35 R45 2008 ENGL 505A
"Colonialism and the emergence of science fiction," Rieder, John, PS374.S35 R45 2008 ENGL 535A
"Metamorphoses of science fiction: on the poetics and history of a literary genre," Suvin, Darko, PN3433.8 .S897 1979 ENGL 505A
"Science fiction before 1900: imagination discovers technology," Alkon, Paul K, PN3433.5 .A45 1994 ENGL 505A
"Science Fiction: A Literary History," Luckhurst, Roger (ed.), PN3433.8 .S353 2017 ENGL 505A
"Victorian science fiction in the UK: the discourses of knowledge and of power," Suvin, Darko, PR878.S35 S8 1983 ENGL 505A


### Define Function: Search by Column/Field
This function helps you locate all entries that contain a certain value in a particular field/column.

Matches can be partial - eg. 'Automat' with return entries with 'automatic','automaton', 'automata', etc.

In [12]:
def search_by_column(colname,searchstr):
  import re
  results = []
  for ind in range(0,len(df)):
    bFound = False
    entry = df.loc[ind]
    test = re.search(searchstr,str(entry[colname]))
    if not test == None:
      headers = ['Title','Author','Course code', 'Coursenumber','Call number']
      if not colname in headers:
        headers.append(colname)

      citation = Record(entry.index.values,entry.to_list())
      citation.print('brief')
      results.append(citation)

  return


def find_by_column(colname,searchstr):
  results = []
  for ind in range(0,len(df)):
    bFound = False
    entry = df.loc[ind]
    if entry[colname]== searchstr:
      results.append(entry)
  return results

### Search by Field or Column Value
Refer to the Column Names (above) for a list of possible fields to search. This function requires a field/column and a value or list of values to match.

In this example, we're searching for call numbers that begin with PR878, PS374, PS648, and PN3433.

In [13]:
search_by_column('Call number','PR878|PS374|PS648|PN3433')



"Archaeologies of the future: the desire called utopia and other science fictions," Jameson, Fredric, PS648.S3 J36 2007 ENGL 505A
"Colonialism and the emergence of science fiction," Rieder, John, PS374.S35 R45 2008 ENGL 505A
"Colonialism and the emergence of science fiction," Rieder, John, PS374.S35 R45 2008 ENGL 535A
"Genreflecting: A guide to popular reading interests (8th edition).," Herald, Diana Tixier; Stavole-Carter, Samuel, PS374.P63 H47 2019 LIBR 545
"Metamorphoses of science fiction: on the poetics and history of a literary genre," Suvin, Darko, PN3433.8 .S897 1979 ENGL 505A
"Science fiction before 1900: imagination discovers technology," Alkon, Paul K, PN3433.5 .A45 1994 ENGL 505A
"Science Fiction: A Literary History," Luckhurst, Roger (ed.), PN3433.8 .S353 2017 ENGL 505A
"Telegraphic realism," Menke, Richard, PR878.C636 M46 2008 ENGL 505A
"Victorian science fiction in the UK: the discourses of knowledge and of power," Suvin, Darko, PR878.S35 S8 1983 ENGL 505A


### Define Function: Search by Course Number
This function enables you to search for entries related to a particular course number. Because course code and course number are different fields, you cannot use the previous function to search for a combination of terms.

We can pass in a course by its full course code and number (eg. LIBR 598) and the function then splits it into its parts in order to find entries which match course codes and course numbers.

In [14]:
def search_by_course_number(courses):
  import re
  results = []
  course_list = courses.split('|')

  for course in course_list:
    course_code,course_num = course.split(' ')

    for ind in range(0,len(df)):
      bFound = False
      entry = df.loc[ind]
      if entry['Course code'] == course_code and entry['Coursenumber'] == course_num:
        headers = ['Title','Author','Course code', 'Coursenumber','Call number']
        #citation = Record(entry.index,entry.to_list)
        citation = ''
        for hd in headers:
          citation = citation +str(entry[hd])+', '
        #citation.print('brief')
        results.append(citation)

  for ind in range(0,len(results)):
    print(results[ind])

  return


def search_by_course_id(courses):
  catalog = Catalog(df)
  results = catalog.search_by_course_id(courses)
  for result in results:
    result.print('brief')

### Search by Course Number
Let's say we know we want to locate materials that have used/reserved for previous times ENGL 505A and ENGL 535A were taught. We can pass in the list of course numbers and get a report.

In [15]:
catalog = Catalog(df)
#search_by_course_number('ENGL 505A|ENGL 535A')
#catalog.search_by_course_id('ENGL 505A|ENGL 535A')
search_by_course_id('ENGL 505A|ENGL 535A')

"Alice's Adventures: Lewis Carroll in Popular Culture," Brooker, Will, PR4611.A73 B75 2004 ENGL 535A
"Archaeologies of the future: the desire called utopia and other science fictions," Jameson, Fredric, PS648.S3 J36 2007 ENGL 505A
"Colonialism and the emergence of science fiction," Rieder, John, PS374.S35 R45 2008 ENGL 505A
"Colonialism and the emergence of science fiction," Rieder, John, PS374.S35 R45 2008 ENGL 535A
"Erewhon," Butler, Samuel;Mudford, Peter, PR4349.B7 E6 1970 ENGL 505A
"George Eliot and nineteenth-century science: the make-believe of a beginning," Shuttleworth, Sally, PR4692.S3S5 1984 ENGL 535A
"George Eliot's serial fiction," Martin, Carol A, PR4688 .M38 1994 ENGL 535A
"Haunted media: electronic presence from telegraphy to television," Sconce, Jeffrey, P96.T42S37 2000 ENGL 505A
"Imperial media: colonial networks and information technologies in the British literary imagination, 1857-1918," Worth, Aaron, PR468.T4 W67 2014 ENGL 505A
"Imperial media: colonial networks and

## Generate Reports

### Book/Title Report


In [16]:
def get_book_report(book_title='*',bShow=True):
  catalog = Catalog(df)
  if book_title == '*':
    results = catalog.get()
  else:
    results = catalog.search('Title',book_title)

  reports = []

  if results == []:
    print(f'No texts titled {book_title} found.')
    return reports

  results.sort(key=lambda x: str(x.Title))

  last_title = ''
  bFound = False

  for entry in results:
    if reports == []:
      report = {'title':entry.Title,'count':1,'courses':[entry.course_id],'sessions':[entry.Session]}
      reports.append(report)
    else:
      bFound = False
      for rep in reports:
        if entry.Title == rep['title']:
          rep['count'] = rep['count'] + 1
          rep['courses'].append(entry.course_id)
          rep['sessions'].append(entry.Session)
          bFound = True

      if bFound == False:
        report = {'title':entry.Title,'count':1,'courses':[entry.course_id],'sessions':[entry.Session]}
        reports.append(report)
  if bShow == True:
    for rep in reports:
      print(f'\nTitle: {rep["title"]}\n Count: {rep["count"]}\n Courses: {rep["courses"]}\n Sessions: {rep["sessions"]}')
  return reports


In [17]:
reports = get_book_report("Science Fiction")


Title: Archaeologies of the future: the desire called utopia and other science fictions
 Count: 1
 Courses: ['ENGL 505A']
 Sessions: ['2023W2']

Title: Colonialism and the emergence of science fiction
 Count: 2
 Courses: ['ENGL 505A', 'ENGL 535A']
 Sessions: ['2023W2', '2021W1']

Title: Metamorphoses of science fiction: on the poetics and history of a literary genre
 Count: 1
 Courses: ['ENGL 505A']
 Sessions: ['2023W2']

Title: Science Fiction: A Literary History
 Count: 1
 Courses: ['ENGL 505A']
 Sessions: ['2023W2']

Title: Science fiction before 1900: imagination discovers technology
 Count: 1
 Courses: ['ENGL 505A']
 Sessions: ['2023W2']

Title: Victorian science fiction in the UK: the discourses of knowledge and of power
 Count: 1
 Courses: ['ENGL 505A']
 Sessions: ['2023W2']


### Other Questions
- What's been assigned by who
- List of departments using LOCR
- Publisher data - Which publishers (counts)
- How often does a text get assigned


### Concerns
- This dataset would be stronger if syllabus service was more actively used (because texts would automatically be added to LOCR)

In [18]:
def get_publisher_report(searchterm, bShow=True):
  catalog = Catalog(df)
  results = catalog.search('Publisher',searchterm)

  reports = []

  if results == []:
    print(f'No texts with {searchterm} as Publisher found.')
    return

  results.sort(key=lambda x: getattr(x,'Publisher'))

  last_title = ''
  bFound = False

  for entry in results:
    if reports == []:
      report = {'titles':[entry.Title],'count':1,'publisher':entry.Publisher}
      reports.append(report)
    else:
      bFound = False
      for rep in reports:
        if entry.Publisher == rep['publisher']:
          rep['count'] = rep['count'] + 1
          rep['titles'].append(entry.Title)
          bFound = True

      if bFound == False:
        report = {'titles':[entry.Title],'count':1,'publisher':entry.Publisher}
        reports.append(report)
  if bShow:
    for rep in reports:
      print(f"\nPublisher: {rep['publisher']}\n Count: {rep['count']}\n Titles: {rep['titles']}")

In [19]:
get_publisher_report("harvard")


Publisher: Belknap Press of Harvard University Press
 Count: 3
 Titles: ['A history of the Arab peoples', 'A history of the Arab peoples', 'A history of the Arab peoples']

Publisher: Harvard
 Count: 1
 Titles: ['Empires and the Reach of the Global: 1870Ã¢â‚¬â€œ1945']

Publisher: Harvard University Asia Center
 Count: 1
 Titles: ['The efficacious landscape: on the authorities of painting at the Northern Song court']

Publisher: Harvard University Press
 Count: 29
 Titles: [' Birthright lottery: citizenship and global inequality', "China's response to the West: a documentary survey, 1839-1923 ", 'Continental divide: Heidegger, Cassirer, Davos', "Opium's long shadow: from Asian revolt to global drug control", 'Oversold and underused: computers in the classroom', 'Oversold and underused: computers in the classroom', 'Oversold and underused: computers in the classroom', 'Oversold and underused: computers in the classroom', 'Oversold and underused: computers in the classroom', 'Oversold an

In [20]:
def get_reserve_rankings(num_entries=10):
  catalog = Catalog(df)
  results = get_book_report(' ',False)
  results.sort(key=lambda x: int(x['count']),reverse=True)
  max = num_entries
  if max > len(results):
    max = len(results)
  i = 0
  for rep in results:
    if (i<max):
      print(f'\nTitle: {rep["title"]}\n Count: {rep["count"]}\n Courses: {rep["courses"]}\n Sessions: {rep["sessions"]}')
      i = i+1
    else:
      return

In [21]:
get_reserve_rankings()


Title: Teachers Doing Research: The Power of Action Through Inquiry
 Count: 99
 Courses: ['EDUC 490I', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450B', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A', 'EDUC 450A',

In [22]:
def get_dept_report(searchterm='*',bShow=True):
  catalog = Catalog(df)
  results = catalog.search('Course code',searchterm)

  reports = []

  if results == []:
    print(f'No texts with {searchterm} as Course code found.')
    return

  results.sort(key=lambda x: getattr(x,'Course code'))
  print(results)
  last_title = ''
  bFound = False

  for entry in results:
    if reports == []:
      report = {'titles':[entry.Title],'count':1,'course':getattr(entry,'Course code')}
      reports.append(report)
    else:
      bFound = False
      for rep in reports:
        if getattr(entry,'Course code') == rep['course']:
          rep['count'] = rep['count'] + 1
          rep['titles'].append(entry.Title)
          bFound = True

      if bFound == False:
        report = {'titles':[entry.Title],'count':1,'course':getattr(entry,'Course code')}
        reports.append(report)
  if bShow:
    for rep in reports:
      print(f"\nCourse: {rep['course']}\n Count: {rep['count']}\n Titles: {rep['titles']}")

  return reports

In [23]:
reports = get_dept_report('*')

[<__main__.Record object at 0x78f967cad930>, <__main__.Record object at 0x78f967cf6c80>, <__main__.Record object at 0x78f967cf7eb0>, <__main__.Record object at 0x78f980979270>, <__main__.Record object at 0x78f967cae530>, <__main__.Record object at 0x78f967cae4a0>, <__main__.Record object at 0x78f967cc78b0>, <__main__.Record object at 0x78f967cc7880>, <__main__.Record object at 0x78f967cc7850>, <__main__.Record object at 0x78f967cc7820>, <__main__.Record object at 0x78f967cc6e60>, <__main__.Record object at 0x78f99f224b50>, <__main__.Record object at 0x78f967caf340>, <__main__.Record object at 0x78f967caded0>, <__main__.Record object at 0x78f967caf7c0>, <__main__.Record object at 0x78f967caf880>, <__main__.Record object at 0x78f967caf610>, <__main__.Record object at 0x78f967caf520>, <__main__.Record object at 0x78f967caf5e0>, <__main__.Record object at 0x78f967cc4400>, <__main__.Record object at 0x78f980375f90>, <__main__.Record object at 0x78f967cae3b0>, <__main__.Record object at 0x78

In [ ]:
def get_dept_ranking(num_entries=10):
    catalog = Catalog(df)
    results = get_dept_report('*',False)

    if results == [] or results == None:
      print(f'No texts')
      return

    results.sort(key=lambda x: int(x['count']),reverse=True)
    max = num_entries
    if max > len(results):
      max = len(results)
    i = 0
    for rep in results:
      if (i<max):
        print(f'\nCourse: {rep["course"]}\n Count: {rep["count"]}\n Titles: {rep["titles"]}')
        i = i+1
      else:
        return

In [ ]:
get_dept_ranking()

[<__main__.Record object at 0x7f0a8b509180>, <__main__.Record object at 0x7f0a57a22e60>, <__main__.Record object at 0x7f0a57a23fd0>, <__main__.Record object at 0x7f0a57ac6da0>, <__main__.Record object at 0x7f0a8b508af0>, <__main__.Record object at 0x7f0a8b5098a0>, <__main__.Record object at 0x7f0a6446a6e0>, <__main__.Record object at 0x7f0a6446a710>, <__main__.Record object at 0x7f0a6446a740>, <__main__.Record object at 0x7f0a6446a770>, <__main__.Record object at 0x7f0a6446b220>, <__main__.Record object at 0x7f0a57aed7b0>, <__main__.Record object at 0x7f0a8b508550>, <__main__.Record object at 0x7f0a8b50be20>, <__main__.Record object at 0x7f0a8b509210>, <__main__.Record object at 0x7f0a8b5091e0>, <__main__.Record object at 0x7f0a8b508160>, <__main__.Record object at 0x7f0a8b5092d0>, <__main__.Record object at 0x7f0a8b508220>, <__main__.Record object at 0x7f0a64469a50>, <__main__.Record object at 0x7f0a57a79d20>, <__main__.Record object at 0x7f0a8b509000>, <__main__.Record object at 0x7f